# Minilab 3 - Mapping TAZ travel times

In this lab we will play with some data from the Metropolitan Transportation Comission on travel time from one Traffic Analysis Zone (TAZ) to another. We will use a mapping tool called folium to create a graphical representation of travel times throughout the area.


In [ ]:
import folium
import json
from datascience import *
import pandas as pd

## The datasets
### MTC travel skims
The Metropolitan Transportation Comission (MTC) is the regional transportation planning organization for the Bay Area. They host a database with average travel time, cost, and distance from one traffic analysis zone (TAZ) to all other TAZs in the Bay Area. The files have data for driving alone, car pooling, walking to transit, driving to transit, walking, and biking. 

We have pre-processed the data from the morning commute to include only TAZs around San Francisco, Oakland and Berkeley. The files with inter-TAZ travel time, travel cost, and travel distance are saved with the following filepaths 'data/sf_oak_TimeSkims_AM.csv', 'data/sf_oak_CostSkims_AM.csv' and 'data/sf_oak_DistanceSkims_AM.csv'

More info on the dataset can be found here - http://analytics.mtc.ca.gov/foswiki/Main/SimpleSkims. 
The columns in the data set are below:

    orig	Origin transportation analysis zone	Shape file
    dest	Destination transportation analysis zone	Shape file
    da	    Door-to-door time for the drive alone travel mode (i.e. single occupant private automobile)	 
    daToll	Door-to-door time for the drive alone value toll travel mode (those willing to pay to use an HOT lane)	 
    s2	    Door-to-door time for the shared ride 2 travel mode (i.e. double occupant private automobile)	 
    s2Toll	Door-to-door time for the shared ride 2 value toll travel mode (those willing to pay to use an HOT lane)
    s3	    Door-to-door time for the shared ride 3+ travel mode (i.e. three-or-more occupants traveling in a private vehicle)	 
    s3Toll	Door-to-door time for the shared ride 3+ value toll travel mode (those willing to pay to use an HOT lane, if scenario policy dictates they must pay)	 
    walk	Door-to-door time for walking	 
    bike	Door-to-door time for bicycling	 
    wTrnW	Door-to-door time for walk to transit to walk paths	 
    dTrnW	Door-to-door time for drive to transit to walk paths	 
    wTrnD	Door-to-door time for walk to transit to drive paths (returning home on a park-and-ride tour)

(The raw data with all bay area TAZs can be found at https://mtcdrive.app.box.com/2015-03-116)

### Bay area TAZ geometry data
GeoJSON is a format for encoding a variety of geographic data structures. We have saved a GeoJson file with the Trffic Analysis Zone (TAZ) boundaries for the TAZs around San Francisco, Oakland, and Berkeley. We will use a mapping package called folium to map the TAZs

### Read the data
Normally we use the Table function to read the dataset, but for this lab we are using a mapping tool and we require the data to be in a Pandas Dataframe. The functionality is **very** similar to the table functionality.

In [ ]:
# Read the timeskims datafile
data = pd.read_csv('data/sf_oak_TimeSkims_AM.csv')
#print out the first 5 rows of the data table
data.head()

In [ ]:
# Read the GeoJson file save the data as a string
# json.dumps() dumps the data to a string
geojson_data = json.load(open('data/SF_Oak_TAZs.geojson'))
geojson_str = json.dumps(geojson_data) 

# The geojson string contains properties of the TAZ as well as lat-lon coordinates defining 
# the polygon boundary for each TAZ. Here is a printout of what the geojson looks like.
geojson_str

## Mapping the TAZs
We will use a mapping package called Folium to map the TAZs



In [ ]:
# Create a folium map object. Set the width, and height of the map image 
# as well as start zoom level and center location
taz_map = folium.Map(width=650, height=500, zoom_start=11,
                     location=[37.8062449,-122.371983])

# Add the taz geojson data layer to the map using the geo_json() method
taz_map.geo_json(geo_str=geojson_str)

# Create the map - notice that a file called map.html has been added to your demos folder 
taz_map.create_map('map.html')

#to show the map in-line
taz_map

## Travel time from one TAZ to all others


In [ ]:
#choose an origin TAZ
origin_taz_id=10

# save a new data frame that contains only rows with the origin specified above 
data_origin = data[data['orig']==origin_taz_id]

Folium allows us to color map the elemets based on data. We will again use folium's geo_json() method, but now we will use additional arguments to color the TAZs based on travel time. 
We will use the following inputs to the geojson method:

    data: Pandas DataFrame or Series, default None
        Data to bind to the GeoJSON.
    columns: the columns of the dataframe that we will use to color the data
    key_on: string, default None
        Variable in the GeoJSON file to bind the data to. Must always
        start with 'feature' and be in JavaScript objection notation.
        Ex: 'feature.id' or 'feature.properties.statename'. 
    threshold_scale: list, default None
        Data range for D3 threshold scale. Defaults to the following range
        of quantiles: [0, 0.5, 0.75, 0.85, 0.9], rounded to the nearest
        order-of-magnitude integer. Ex: 270 rounds to 200, 5600 to 6000.
    fill_color: string, default 'blue'
        Area fill color. Can pass a hex code, color name, or if you are
        binding data, one of the following color brewer palettes:
        'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu',
        'YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.


In [ ]:
traveltime_map = folium.Map(width=650, height=500, zoom_start=11,
                            location=[37.8062449,-122.371983])

traveltime_map.geo_json(geo_str=geojson_str, data=data_origin,
                        columns=['dest', 'da'],
                        threshold_scale=[5,10,15,20,25],
                        key_on='feature.properties.TAZ1454',
                        fill_color='PuBu', fill_opacity=.8)

traveltime_map.create_map('map.html')
traveltime_map



## Color the origin TAZ red

In [ ]:
# find origin_taz_id and color it red
for i in range(len(geojson_data['features'])):
    if (geojson_data['features'][i]['properties']['TAZ1454'] == origin_taz_id):
        origin_feature  = geojson_data['features'][i]

# create a copy of the geojson data called origin_data. This geojson will only have 
# one feature, the origin feature. We will replace the origin_data['features'] with
# a list containing only this feature

origin_data = geojson_data
origin_data['features']=[origin_feature]
origin_str = json.dumps(origin_data)

traveltime_map.geo_json(geo_str=origin_str,fill_color='Red',
                       fill_opacity = .8)

traveltime_map.create_map('map.html')
traveltime_map

# Task 1
Play with the map. 
 <li>Change the map color scheme.
 <li>Choose a different TAZ as the origin TAZ
 <li>Right now average drive alone (da) travel times are displayed. Change the travel mode to walk-transit-walk (wTrnW)
 <li>Change the color threshold values so that the color transitions happen at appropriate intervals

# Task 2
What can we learn from these maps
For each of the following modes: drive, walk to transit, drive to transit, bike, and walk, do you think a straightline distance from one TAZ to another is a good indicator of travel time? Explain why or why not?



In [ ]:
# Answer here

# Task 3
For the datasets provided, MTC follows a convention of setting travel time = '-999' in the data table if they have determined that a mode is not feasible. For example, a bike trip from San Francisco to Oakland is not possible because bikes are not allowed accross the Bay Bridge. Similarly some walk distances are determined to be too far to make a trip on foot.

Can you figure out how to find the entries that have -999 in the travel time field and make them grey?


# Task 4
We have been looking at travel time from one TAZ to another. As mentioned above MTC also provides info on travel cost and distance from one TAZ to another by mode. Create a folium map of tavel distance from an origin TAZ to all others, using the same procedure as outlined above.